In [ ]:
import sys
sys.path.append('/home/adria/lung_cancer_isbi18/src/dl_networks/')
import os
import random
import logging
# import multiprocessing
import numpy as np
import pandas as pd
# from time import time
# import matplotlib.pyplot as plt
# from utils import plotting
import  common
# from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
import sample_resnet_malignancy 
# from dl_utils.tb_callback import TensorBoard

In [ ]:
# PATHS
# wp = os.environ['LUNG_PATH']
wp = '/media/shared/datasets/LUNA/'
INPUT_PATH = '/home/adria/data/'  # INPUT_PATH = wp + 'data/preprocessed5_sample'
VALIDATION_PATH = '/home/adria/data/'
NODULES_PATH = wp + '/annotations_enhanced.csv'
PATCHES_PATH = '/home/adria/data/patches/'  # PATCHES_PATH = wp + 'data/preprocessed5_patches'

OUTPUT_MODEL = '/home/adria/data/models/'  # OUTPUT_MODEL = wp + 'personal/jm_patches_train_v06_local.hdf5'
LOGS_PATH = wp + 'logs/%s' % 'patches_v1'
if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)



## PATCHES GENERATION -----------------------------------------------------------------

## PATIENTS FILE LIST
patients_with_annotations = pd.read_csv(NODULES_PATH)  # filter patients with no annotations to avoid having to read them
patients_with_annotations = list(set(patients_with_annotations['seriesuid']))
patients_with_annotations = ["luna_%s.npz" % p.split('.')[-1] for p in patients_with_annotations]

filenames = os.listdir(INPUT_PATH)
filenames = [g for g in filenames if g.startswith('luna_')]
filenames_train = [os.path.join(INPUT_PATH, fp) for fp in filenames if fp in patients_with_annotations]
filenames_test = [os.path.join(VALIDATION_PATH, fp) for fp in os.listdir(VALIDATION_PATH) if fp in patients_with_annotations]


def __load_and_store(filename):
    patient_data = np.load(filename)['arr_0']
    X, y = common.load_patient_malignancy(patient_data, discard_empty_nodules=True, thickness=1)
    logging.info("Patient: %s" % (filename.split('/')[-1]))
    return X, y

common.crop_generator_malignancy(filenames_train,
                                os.path.join(PATCHES_PATH, 'dl1_v1_x_train.npz'),
                                os.path.join(PATCHES_PATH, 'dl1_v1_y_train.npz'),
                                __load_and_store)



In [ ]:
def crop_generator_malignancy(filenames, out_x_filename, out_y_filename, load_patient_func, store=True):

    xf,yf = [],[]
    for idx, filename in enumerate(filenames):
#         logging.info("Loading %d/%d %" (idx,len(filenames)))
        x,y = load_patient_func(filename)
        xf.extend(x)
        yf.extend(y)
        
#     logging.info('Total time: %.2f, total patients:%d' % (time() - tstart, len(x)))
    if store:
        np.savez_compressed(out_x_filename, np.asarray(xf))
        np.savez_compressed(out_y_filename, np.asarray(yf))
        logging.info('Finished saving files')

In [ ]:
x,y = load_patient_malignancy(patient_data,discard_empty_nodules=True, thickness = 1)

In [ ]:
xf = []
xf.extend(X)


In [ ]:
xf

In [ ]:
x_train = np.load(os.path.join(PATCHES_PATH, 'dl1_v1_x_train.npz'))['arr_0']

In [ ]:
x_train.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[4][2])
plt.show()